# Doc2Vec

In [5]:
!pip install sctokenizer
!pip install nltk
!pip install scipy==1.12
!pip install gensim
!pip install typing


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/78.6 kB ? eta -:--:--
     -------------------- ------------------- 41.0/78.6 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 78.6/78.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26327 sha256=1c82015a6c82c957b2a1d30f1c6fd5635bd6d0f3d1963064dcb618304e113b68
  Stored in directory: c:\users\migue\appdata\local\pip\cache\wheels\7c\d0\9e\1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from sctokenizer import JavaTokenizer, TokenType
import math
def tokenize_code(code):
    """
    Tokenize Java code using JavaTokenizer from sctokenizer.

    Parameters:
    code (str): The Java code to tokenize.

    Returns:
    list: A list of token types or token values.
    """

    tokenizer = JavaTokenizer()
    tokens = tokenizer.tokenize(code)
    token_list = []
    for token in tokens:
      if token.token_type == TokenType.COMMENT_SYMBOL:
        continue
      if token.token_type == TokenType.IDENTIFIER:
        token_list.append(token.token_type)
      else:
        token_list.append(token.token_value)

    return token_list

In [41]:
import re

class Document:
  """
  Represents a document object for storing and processing text content.
  """
  all_documents = []

  def __init__(self, doc_name: str) -> None:
    """
    Initializes a Document object with the provided document name.

    Args:
        doc_name (str): The name of the document file.
    """
    self.doc_name = doc_name
    self.word_dict: dict[str, int] = {}
    self.raw_text = ""
    self.text = ""
    self.tokens: list[str] = []
    self.raw_comments: str = ""
    self.token_comments: list[str] = []
    self.one_line_text: str = ""

    self.__read_text()
    self.__create_word_dict()
    self.__get_comments(self.raw_text)
    self.__get_one_line_text()
    Document.all_documents.append(self)

  def __get_comments(self, code):
    """
    Extracts comments from the provided code string.

    Args:
        code (str): The code string to extract comments from.
    """
    single_line_comment_pattern = r'//.*'
    multi_line_comment_pattern = r'/\*[\s\S]*?\*/'

    single_line_comments = re.findall(single_line_comment_pattern, code)
    multi_line_comments = re.findall(multi_line_comment_pattern, code)

    stripped_single_line_comments = [comment.lstrip('//').strip() for comment in single_line_comments]
    stripped_multi_line_comments = [re.sub(r'(^/\*|\*/$)', '', comment).strip() for comment in multi_line_comments]

    all_comments = ""
    for comment in stripped_single_line_comments:
      all_comments += f"{comment} "

    for comment in stripped_multi_line_comments:
      all_comments += f"{comment}"

    self.raw_comments = all_comments
    self.token_comments = self.raw_comments.split(" ")

  def __read_text(self) -> None:
    """
    Reads the text content from the document file.
    """
    raw_text = ""
    text = ""
    file = open(f"{self.doc_name}", "r", encoding="utf-8")
    while True:
      line = file.readline()
      if not line:
        break

      raw_text += line
      tokenized_line = tokenize_code(line)
      for token in tokenized_line:
        self.tokens.append(str(token))

      text += " ".join(str(tokenized_line))
    file.close()

    self.raw_text = raw_text
    self.text = text

  def __create_word_dict(self):
    """
    Creates a dictionary to store the frequency of each word in the document.
    """
    for token in self.tokens:
      self.word_dict[token] = self.word_dict.get(token, 0) + 1
  
  def __get_one_line_text(self):
    str_tokens = []
    for token in self.tokens:
      str_tokens.append(str(token))

    self.one_line_text = " ".join(str_tokens)

In [42]:
import os
import shutil

def generate_documents(class_name: str, directory: str = "") -> list[dict[str, float]]:
    """
    Generate a table of document comparison metrics for a given class name.

    Parameters:
    class_name (str): The name of the class.

    Returns:
    list: A list of dictionaries containing comparison metrics for each document pair.
    """

    directorio_principal = directory

    data = []

    ruta_clase = os.path.join(directorio_principal, class_name)

    if os.path.exists(ruta_clase):
        for carpeta_id in os.listdir(ruta_clase):
            ruta_carpeta = os.path.join(ruta_clase, carpeta_id)

            if os.path.isdir(ruta_carpeta):
                for archivo in os.listdir(ruta_carpeta):
                    ruta_archivo = os.path.join(ruta_carpeta, archivo)
                    Document(ruta_archivo)
                    
    else:
        print(f'La ruta {ruta_clase} no existe')

    return data

In [43]:
dir_name = "data_set_splitted/"

generate_documents("no_plagiado", f"{dir_name}train")
generate_documents("plagiado", f"{dir_name}train")
generate_documents("no_plagiado", f"{dir_name}val")
generate_documents("plagiado", f"{dir_name}val")
generate_documents("no_plagiado", f"{dir_name}test")
generate_documents("plagiado", f"{dir_name}test")
generate_documents("no_plagiado", "queries")


[]

In [47]:
print(len(Document.all_documents))

data = []

for document in Document.all_documents:  
  data.append(document.tokens)


1832


In [72]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

vector_size = 100
epochs = 50

tagged_data = [TaggedDocument(words=code, tags=[str(i)]) for i, code in enumerate(data)]

model = Doc2Vec(vector_size=vector_size, epochs=epochs)

model.build_vocab(tagged_data)

model.train(tagged_data, 
  total_examples=model.corpus_count,
  epochs=model.epochs
)


In [73]:
model.save("doc_2_vec_model")

In [74]:
code_vectors = []
for code in data:
  code_vectors.append(model.infer_vector(code))

print(code_vectors[0])

[ 0.9038807   0.6516178   0.38068998 -0.21664296 -0.02243141 -0.06655913
 -0.02820565 -0.02017074  0.10825096 -0.58535016  0.5246618   0.11500695
 -0.21995214 -0.10347764  0.08916871 -0.71791947 -0.16580206 -0.32584137
  0.1914409   0.22244132  0.03306208  0.32970122 -0.01666521  0.04224354
 -0.9864555  -0.28224176  0.49276832  0.4007685   0.13869038 -0.5953549
  0.8812585  -1.027326   -0.53101987  0.3375814  -0.3416     -0.32526255
  1.031309    0.34625775 -0.58447295  0.45306736 -0.31948668 -0.0484423
 -0.26935917  0.8983924  -0.2337651   0.5975756   0.14304486  0.3778272
 -0.03943252 -0.40287146 -0.5738162   0.07660487  0.36672214 -0.3848258
 -0.38750222  0.38827497  0.31332833  0.42055115 -0.27143547 -0.40805867
 -0.59332156 -0.24029462  0.67487216 -0.3452443  -0.1233667  -0.29811016
  0.04851728 -0.6381121  -0.16599202 -0.93818843 -0.45292136  0.16069964
  0.6289938   0.18703412 -0.20723057  0.7109115  -0.00558115 -0.19444495
 -0.68056464 -0.41690558 -0.05172285  0.12327626 -0.208